<h1 align=center> Proceso de ML <sub align=center> (ML PROCESS)</sub></h1>

<h3 align=left> Instalacion de librerias<sub>/Library installation</sub> </h3>

In [ ]:
pip install -r requirements.txt

<h3 align=left> Importacion de librerias<sub>/Library Importing</sub> </h3>

In [47]:
import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


In [65]:
data = pd.read_parquet('../data_steam.parquet')
data_steam_games = data.copy()
steam = data.copy()

In [49]:
data_steam_games.drop(['id','user_id','developer','genre','year','sentiment_analysis','recommend','price','playtime_forever','release_year'],axis=1,inplace=True)
data.drop(['id','user_id','genre','year','sentiment_analysis','recommend','price','playtime_forever'],axis=1,inplace=True)

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32605 entries, 0 to 32604
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   item_id       32605 non-null  object
 1   title         32605 non-null  object
 2   developer     32605 non-null  object
 3   release_year  32605 non-null  int32 
dtypes: int32(1), object(3)
memory usage: 891.7+ KB


In [15]:
data_steam_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32605 entries, 0 to 32604
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   item_id  32605 non-null  object
 1   title    32605 non-null  object
dtypes: object(2)
memory usage: 509.6+ KB


In [50]:
data['item_id'] = data['item_id'].astype(str)
data['release_year'] = data['release_year'].astype(str)

# borrando duplicados
data = data.drop_duplicates(subset='item_id', keep='first')
data_steam_juegos = data_steam_games.drop_duplicates(subset='item_id', keep='first')

# Combinar los valores de las columnas en una sola columna
data['specs'] = data[["title", "developer", "release_year"]].apply(lambda x: ", ".join(x), axis=1)
data.drop(['title','developer','release_year'],axis=1,inplace=True)


Conversion a parquet

In [51]:
data.to_parquet('steam_games.parquet', index=False)
data_steam_games.to_parquet('steam_id.parquet', index=False)

Modelo de recomendacion

In [67]:
#Seleccionar características (X) y la etiqueta (y)
X = steam[["item_id", "release_year", "price", "sentiment_analysis", "playtime_forever"]]
y = steam["recommend"]

#Divide el conjunto de datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Crea y entrena el modelo
modelo = RandomForestClassifier(n_estimators=100, random_state=42)
modelo.fit(X_train, y_train)

#Realiza predicciones en el conjunto de prueba
predicciones = modelo.predict(X_test)

#Evalúa la precisión del modelo
precision = accuracy_score(y_test, predicciones)
print(f"Precisión del modelo: {precision}")

Precisión del modelo: 0.9007820886367122
